# Projet : Analysons des factures d'énergie 

In [3]:
import pandas as pd
import numpy as np
import re

In [65]:
invoice = ('Invoice_20201220.csv')
df = pd.read_csv(invoice, delimiter='|', header=0)
print(df.shape)
display(df)

(51432, 6)


,SITE_NAME,INVOICE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,12345_GEO (0001BB),1300082549,01/01/2018,31/01/2018,2 447,"316,46"
1,12345_GEO (0001BB),1300082549,01/02/2018,28/02/2018,2 211,"286,82"
2,12345_GEO (0001BB),1300082549,01/03/2018,31/03/2018,2 450,"317,32"
3,12345_GEO (0001BB),1300082549,01/04/2018,30/04/2018,2 431,"315,00"
4,12345_GEO (0001BB),1300082549,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...,...
51427,19767_GEO (0632KO),FV1/055100030,30/08/2019,31/12/2019,3 028,"397,18"
51428,19767_GEO (0632KO),055100058,01/01/2020,31/03/2020,2 163,"283,57"
51429,19767_GEO (0632KO),055100113,01/04/2020,30/06/2020,2 160,"312,55"
51430,19767_GEO (0632KO),055100168,01/07/2020,30/09/2020,2 230,"294,07"


### 1) Le nom d'une antenne a un format bien précis : il est toujours composé de 4 chiffres et de deux lettres. On te demande de retirer tout autre caractère de la colonne "SITE_NAME".

In [66]:
#donne le format que je veux garder : 4 digits, _ et 2 words
df['SITE_NAME'] = df['SITE_NAME'].str.extract('(\d{4}_\w{2})', expand=False) 
print(df)

      SITE_NAME   INVOICE_NAME       BEGIN         END KWH_PERIOD PERIOD_EUR
0       2345_GE     1300082549  01/01/2018  31/01/2018      2 447     316,46
1       2345_GE     1300082549  01/02/2018  28/02/2018      2 211     286,82
2       2345_GE     1300082549  01/03/2018  31/03/2018      2 450     317,32
3       2345_GE     1300082549  01/04/2018  30/04/2018      2 431     315,00
4       2345_GE     1300082549  01/05/2018  31/05/2018      2 574     333,37
...         ...            ...         ...         ...        ...        ...
51427   9767_GE  FV1/055100030  30/08/2019  31/12/2019      3 028     397,18
51428   9767_GE      055100058  01/01/2020  31/03/2020      2 163     283,57
51429   9767_GE      055100113  01/04/2020  30/06/2020      2 160     312,55
51430   9767_GE      055100168  01/07/2020  30/09/2020      2 230     294,07
51431   9883_GE     7220278453  27/09/2019  31/12/2019      4 745     681,96

[51432 rows x 6 columns]


## 2) On te demande de supprimer la colonne "INVOICE_NAME" qui n'apporte rien aux équipes.

In [67]:
#Supprimme la colonne "INVOICE_NAME"
df = df.drop(columns=["INVOICE_NAME"])
display(df)

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR
0,2345_GE,01/01/2018,31/01/2018,2 447,"316,46"
1,2345_GE,01/02/2018,28/02/2018,2 211,"286,82"
2,2345_GE,01/03/2018,31/03/2018,2 450,"317,32"
3,2345_GE,01/04/2018,30/04/2018,2 431,"315,00"
4,2345_GE,01/05/2018,31/05/2018,2 574,"333,37"
...,...,...,...,...,...
51427,9767_GE,30/08/2019,31/12/2019,3 028,"397,18"
51428,9767_GE,01/01/2020,31/03/2020,2 163,"283,57"
51429,9767_GE,01/04/2020,30/06/2020,2 160,"312,55"
51430,9767_GE,01/07/2020,30/09/2020,2 230,"294,07"


### 3) Il y a des antennes pour lesquelles on a des factures mensuelles tandis que pour d'autres, on les reçoit moins régulièrement (trimestre, semestre, année). Ajoute une colonne à ton tableau qui calcule le nombre de jours entre le début de la période de facturation et la fin de la période de facturation.

In [69]:
# Convertit les colonnes BEGIN et END en objets datetime
df['BEGIN'] = pd.to_datetime(df['BEGIN'], format='%d/%m/%Y')
df['END'] = pd.to_datetime(df['END'], format='%d/%m/%Y')

# Calcule le nombre de jours entre les colonnes BEGIN et END
df['DAYS'] = (df['END'] - df['BEGIN']).dt.days

#Rajoute a nouvelle colonne DAYS dans le df
df = df.assign(DAYS=(df['END'] - df['BEGIN']).dt.days)
display(df)

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS
0,2345_GE,2018-01-01,2018-01-31,2 447,"316,46",30
1,2345_GE,2018-02-01,2018-02-28,2 211,"286,82",27
2,2345_GE,2018-03-01,2018-03-31,2 450,"317,32",30
3,2345_GE,2018-04-01,2018-04-30,2 431,"315,00",29
4,2345_GE,2018-05-01,2018-05-31,2 574,"333,37",30
...,...,...,...,...,...,...
51427,9767_GE,2019-08-30,2019-12-31,3 028,"397,18",123
51428,9767_GE,2020-01-01,2020-03-31,2 163,"283,57",90
51429,9767_GE,2020-04-01,2020-06-30,2 160,"312,55",90
51430,9767_GE,2020-07-01,2020-09-30,2 230,"294,07",91


### 4) A partir de cette nouvelle colonne, crée une autre colonne qui nous donne la consommation journalière moyenne de la période.


In [88]:
#Enlève les espaces entre les chiffres et transforme le format de 'KWH_PERIOD' en float, car le type était string
df['KWH_PERIOD'] = df['KWH_PERIOD'].replace(' ', '').fillna(np.nan).astype(float)

#Division de la conso par le nb de jours
df ['MEAN_DAYS'] = (df['KWH_PERIOD'] / df['DAYS'])
print(df)

      SITE_NAME      BEGIN        END  KWH_PERIOD PERIOD_EUR  DAYS  MEAN_DAYS
0       2345_GE 2018-01-01 2018-01-31      2447.0     316,46    30  81.566667
1       2345_GE 2018-02-01 2018-02-28      2211.0     286,82    27  81.888889
2       2345_GE 2018-03-01 2018-03-31      2450.0     317,32    30  81.666667
3       2345_GE 2018-04-01 2018-04-30      2431.0     315,00    29  83.827586
4       2345_GE 2018-05-01 2018-05-31      2574.0     333,37    30  85.800000
...         ...        ...        ...         ...        ...   ...        ...
51427   9767_GE 2019-08-30 2019-12-31      3028.0     397,18   123  24.617886
51428   9767_GE 2020-01-01 2020-03-31      2163.0     283,57    90  24.033333
51429   9767_GE 2020-04-01 2020-06-30      2160.0     312,55    90  24.000000
51430   9767_GE 2020-07-01 2020-09-30      2230.0     294,07    91  24.505495
51431   9883_GE 2019-09-27 2019-12-31      4745.0     681,96    95  49.947368

[51432 rows x 7 columns]


### 5) Merci d'ajouter également une colonne qui donne, pour chaque antenne, le nombre de factures dont on dispose au total. Pas grave si l'information se répète à chaque ligne pour une même antenne.

In [90]:
# Création d'une nouvelle colonne qui contient le nombre de factures pour chaque antenne
df['NB_INVOICES'] = df.groupby('SITE_NAME')['SITE_NAME'].transform('count')
display(df)

,SITE_NAME,BEGIN,END,KWH_PERIOD,PERIOD_EUR,DAYS,MEAN_DAYS,NB_INVOICES
0,2345_GE,2018-01-01,2018-01-31,2447.0,"316,46",30,81.566667,35
1,2345_GE,2018-02-01,2018-02-28,2211.0,"286,82",27,81.888889,35
2,2345_GE,2018-03-01,2018-03-31,2450.0,"317,32",30,81.666667,35
3,2345_GE,2018-04-01,2018-04-30,2431.0,"315,00",29,83.827586,35
4,2345_GE,2018-05-01,2018-05-31,2574.0,"333,37",30,85.800000,35
...,...,...,...,...,...,...,...,...
51427,9767_GE,2019-08-30,2019-12-31,3028.0,"397,18",123,24.617886,4
51428,9767_GE,2020-01-01,2020-03-31,2163.0,"283,57",90,24.033333,4
51429,9767_GE,2020-04-01,2020-06-30,2160.0,"312,55",90,24.000000,4
51430,9767_GE,2020-07-01,2020-09-30,2230.0,"294,07",91,24.505495,4


### 6) Et information bonus si tu y arrives : les équipes aimeraient beaucoup avoir une liste de toutes les antennes avec, pour chaque antenne, le coefficient de variation de la consommation. Cela permettra d'avoir une idée, pour chaque antenne, de la dispersion de sa consommation d'énergie.


##### Définition du coefficient de variation : 

Le coefficient de variation est utilisé pour mesurer la dispersion ou la variabilité d'un ensemble de données. C'est un outil utile pour comparer des ensembles de données qui ont des unités de mesure différentes ou des moyennes différentes.

Par exemple, imaginons que Frodon ait marché 10 km chaque jour pendant une semaine, tandis que Gandalf ait marché 15 km chaque jour pendant la même période. Si nous voulons comparer la variabilité de leur distance de marche quotidienne, nous pouvons utiliser le coefficient de variation pour normaliser la mesure.

Ainsi, le coefficient de variation permet de rendre des mesures comparables, même si elles ont des échelles différentes. Il est souvent utilisé dans des domaines tels que la finance, la recherche scientifique ou l'analyse de données pour déterminer si les données ont une dispersion élevée ou faible, et pour identifier des tendances et des anomalies.

In [93]:
# grouper les données par antenne
grouped = df.groupby('SITE_NAME')

# calculer l'écart type de la consommation pour chaque groupe
std = grouped['MEAN_DAYS'].std()

# calculer la moyenne de la consommation pour chaque groupe
mean = grouped['MEAN_DAYS'].mean()

# calculer le coefficient de variation en pourcentage pour chaque groupe
cv = std / mean * 100

# afficher le résultat
print(cv)

SITE_NAME
2345_GE     6.606808
2346_GE     5.048534
2347_GE    10.447159
2348_GE     6.684147
2349_GE     8.402121
             ...    
9604_GE          NaN
9618_GE    62.549750
9680_GE          NaN
9767_GE     1.310412
9883_GE          NaN
Name: MEAN_DAYS, Length: 2404, dtype: float64


### 7) Enfin, tu dois exporter ces deux fichiers obtenus pour pouvoir l'envoyer aux équipes de l'opérateur et leur demander si c'est bien cela qu'ils attendaient de toi.

In [94]:
# Exportez le DataFrame en format CSV
#L'argument index=False indique à Pandas de ne pas inclure l'index du DataFrame dans le fichier exporté.
df.to_csv('Factures_energie_0502.csv', index=False)